In [1]:
import pandas as pd
import pickle

In [2]:
data = pd.read_pickle(r'C:\Users\PIYUSH\Downloads\merged_training.pkl')

In [6]:
data['emotions'].unique()

array(['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
      dtype=object)

In [38]:
data=data.iloc[:200000]

In [39]:
data.shape

(200000, 2)

In [40]:
data.columns = ['content', 'sentiments']

In [41]:
data['sentiments'].unique()

array(['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
      dtype=object)

In [42]:
import re
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [43]:
#Making all letters lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))


In [10]:
# # Code to find the top 10,000 rarest words appearing in the data
# freq = pd.Series(' '.join(data['content']).split()).value_counts()[-10000:]
# # Removing all those rarely appearing words from the data
# freq = list(freq.index)
# data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [44]:
data.shape

(200000, 2)

In [45]:
l=0
for i in range(data.shape[0]):
    l=max(len(data.iloc[i]['content'].split(' ')),l)

In [46]:
l

45

enthusiasm=happy=fun
empty=neutral
boredom=sadness

In [47]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


In [48]:
data.shape

(200000, 2)

In [58]:
#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiments.values)
# Splitting into training and testing data in 90:10 ratio

In [59]:
y.shape

(200000,)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data.content.values, y, test_size=0.1, shuffle=True)


In [27]:
y_train.shape

(180000,)

In [28]:
# Extracting TF-IDF parameters
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

In [29]:
X_train_tfidf=X_train_tfidf.toarray()

In [30]:
X_val_tfidf=X_val_tfidf.toarray()

In [31]:
X_val_tfidf=X_val_tfidf.reshape(X_val_tfidf.shape[0],1,X_val_tfidf.shape[1])

In [32]:
X_train_tfidf=X_train_tfidf.reshape(X_train_tfidf.shape[0],1,X_train_tfidf.shape[1])

In [33]:
from keras.utils import to_categorical
y_label=to_categorical(y_train)
val_label=to_categorical(y_val)
y_label.shape

Using TensorFlow backend.


(180000, 6)

In [65]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [35]:
classifier = Sequential()
classifier.add(LSTM(256,input_shape=(1,1000),return_sequences=True))
classifier.add(Dropout(0.5))
classifier.add(LSTM(256))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 256)            1287168   
_________________________________________________________________
dropout (Dropout)            (None, 1, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1542      
_________________________________________________________________
activation (Activation)      (None, 6)                 0         
Total params: 1,814,022
Trainable params: 1,814,022
Non-trainable params: 0
______________________________________________

In [36]:
val_list=[X_val_tfidf,val_label]

In [37]:
classifier.compile( loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [38]:
classifier.fit(X_train_tfidf,y_label,batch_size=120,epochs=15,validation_split=0.1)

Train on 162000 samples, validate on 18000 samples
Epoch 1/15
162000/162000 [==============================] - 49s 302us/sample - loss: 0.5563 - accuracy: 0.7945 - val_loss: 0.3628 - val_accuracy: 0.8583
Epoch 2/15
162000/162000 [==============================] - 42s 257us/sample - loss: 0.3710 - accuracy: 0.8551 - val_loss: 0.3473 - val_accuracy: 0.8572
Epoch 3/15
162000/162000 [==============================] - 41s 251us/sample - loss: 0.3552 - accuracy: 0.8558 - val_loss: 0.3406 - val_accuracy: 0.8576
Epoch 4/15
162000/162000 [==============================] - 47s 293us/sample - loss: 0.3453 - accuracy: 0.8586 - val_loss: 0.3391 - val_accuracy: 0.8576
Epoch 5/15
 38280/162000 [======>.......................] - ETA: 35s - loss: 0.3332 - accuracy: 0.8624- ETA: 37s - l

KeyboardInterrupt: 

In [83]:
y_prob = classifier.predict(X_val_tfidf) 
y_classes = y_prob.argmax(axis=-1)

In [84]:
y_classes

array([2, 2, 3, ..., 2, 1, 5], dtype=int64)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_classes)

0.852

In [24]:
import numpy as np
embeddings = {}
with open(r'G:\data\glove.6B.100d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [99]:
# import pickle
# with open(r'C:\Users\PIYUSH\Desktop\models_dir\glove_dict.pickle', 'wb') as handle:
#     pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [101]:
# with open(r'C:\Users\PIYUSH\Desktop\models_dir\glove_dict.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [51]:
def getOutputEmbeddings(X):
    embedding_matrix_output = np.zeros((X.shape[0],l,100))
    for ix in range(X.shape[0]):
        sent = X['content'].iloc[ix].split()
        for jx in range(len(sent)):
            try:
                embedding_matrix_output[ix][jx] = embeddings[sent[jx].lower()]
            except:
                continue
            
    return embedding_matrix_output

In [52]:
data.shape

(200000, 2)

In [53]:
glove_data=getOutputEmbeddings(data)

In [54]:
glove_data.shape

(200000, 45, 100)

In [55]:
y.shape

(20000,)

array([4, 4, 2, ..., 1, 1, 0], dtype=int64)

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(glove_data, y, test_size=0.1, shuffle=True)

In [61]:
from keras.utils import to_categorical
y_label=to_categorical(y_train)
val_label=to_categorical(y_val)
y_label.shape

(180000, 6)

In [72]:
from tensorflow.keras.models import model_from_json

In [63]:
X_train.shape

(180000, 45, 100)

In [66]:
classifier = Sequential()
classifier.add(LSTM(128,input_shape=(45,100),return_sequences=True))
classifier.add(Dropout(0.5))
classifier.add(LSTM(128))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 45, 128)           117248    
_________________________________________________________________
dropout (Dropout)            (None, 45, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 774       
_________________________________________________________________
activation (Activation)      (None, 6)                 0         
Total params: 249,606
Trainable params: 249,606
Non-trainable params: 0
__________________________________________________

In [67]:
classifier.compile( loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [68]:
classifier.fit(X_train,y_label,batch_size=120,epochs=6)

Train on 180000 samples
Epoch 1/6
180000/180000 [==============================] - 459s 3ms/sample - loss: 0.6669 - accuracy: 0.7529
Epoch 2/6
180000/180000 [==============================] - 610s 3ms/sample - loss: 0.1928 - accuracy: 0.9171
Epoch 3/6
180000/180000 [==============================] - 710s 4ms/sample - loss: 0.1418 - accuracy: 0.9287
Epoch 4/6
180000/180000 [==============================] - 621s 3ms/sample - loss: 0.1265 - accuracy: 0.9319
Epoch 5/6
180000/180000 [==============================] - 465s 3ms/sample - loss: 0.1138 - accuracy: 0.9362
Epoch 6/6
180000/180000 [==============================] - 468s 3ms/sample - loss: 0.1098 - accuracy: 0.9370


In [69]:
model_json = classifier.to_json()
with open(r"C:\Users\PIYUSH\Desktop\model.json", "w") as json_file:
    json_file.write(model_json)

In [70]:
classifier.save_weights(r"C:\Users\PIYUSH\Desktop\model.h5")
print("Saved model to disk")

Saved model to disk


In [82]:
classifier.save(r"C:\Users\PIYUSH\Desktop\models_dir\model.h5")

In [105]:
from tensorflow.keras.models import load_model

In [84]:
loaded_classifier=load_model(r"C:\Users\PIYUSH\Desktop\models_dir\model.h5")

In [93]:
pred=loaded_classifier.predict(X_val[:1])

In [95]:
y_classes = pred.argmax(axis=-1)

In [96]:
y_classes

array([4], dtype=int64)

In [87]:
accuracy_score(y_val,y_classes)

0.93385

In [86]:
y_classes = pred.argmax(axis=-1)

In [73]:
json_file = open(r'C:\Users\PIYUSH\Desktop\model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [74]:
loaded_model.load_weights(r"C:\Users\PIYUSH\Desktop\model.h5")
print("Loaded model from disk")

Loaded model from disk


In [75]:
pred=loaded_model.predict(X_val)

In [78]:
y_classes = pred.argmax(axis=-1)

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_classes)

0.93385

In [120]:
def testing(X):
    embedding_matrix_output = np.zeros((1,l,100))
    sent = X.split()
    for jx in range(len(sent)):
            try:
                embedding_matrix_output[0][jx] = embeddings[sent[jx].lower()]
            except:
                continue
            
    return embedding_matrix_output

In [132]:
arr=testing('whatever happened it was very bad')
arr.shape

(1, 45, 100)

In [133]:
y_prob = classifier.predict(arr) 
y_classes = y_prob.argmax(axis=-1)

In [134]:
y_classes

array([4], dtype=int64)

In [59]:
test=pd.read_table(r'C:\Users\PIYUSH\Downloads\test.txt',header=None,sep=';')

c:\users\piyush\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [65]:
test.shape

(2000, 2)

In [66]:
#Making all letters lowercase
test[0] = test[0].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation, Symbols
test[0] = test[0].str.replace('[^\w\s]',' ')
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
test[0] = test[0].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
test[0] = test[0].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test[0] = test[0].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))


In [67]:
test.columns = ['content', 'sentiments']

In [69]:
test_data=getOutputEmbeddings(test)

In [73]:
lbl_enc = preprocessing.LabelEncoder()
y_test = lbl_enc.fit_transform(test.sentiments.values)

In [79]:
test_prob=classifier.predict(test_data)

In [80]:
test_prob = test_prob.argmax(axis=-1)

In [81]:
accuracy_score(y_test,test_prob)

0.931

In [91]:
test['sentiments'].value_counts()

joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: sentiments, dtype: int64

In [94]:
test['label']=y_test

In [95]:
test

,content,sentiments,prob,label
0,im feeling rather rotten im ambitious right,sadness,4,4
1,im updating blog feel shitty,sadness,4,4
2,never make separate ever want feel like ashamed,sadness,4,4
3,left bouquet red yellow tulip arm feeling slig...,joy,2,2
4,feeling little vain one,sadness,4,4
5,cant walk shop anywhere feel uncomfortable,fear,1,1
6,felt anger end telephone call,anger,0,0
7,explain clung relationship boy many way immatu...,joy,2,2
8,like breathless feeling reader eager see happe...,joy,2,2
9,jest feel grumpy tired pre menstrual probably ...,anger,0,0


In [87]:
test['prob']=test_prob

In [90]:
test['prob'].value_counts()

2    752
4    580
0    252
1    216
3    107
5     93
Name: prob, dtype: int64

In [96]:
from sklearn.metrics import classification_report

In [98]:
print(classification_report(y_test,test_prob))

             precision    recall  f1-score   support

          0       0.97      0.89      0.93       275
          1       0.92      0.88      0.90       224
          2       0.91      0.99      0.95       695
          3       0.99      0.67      0.80       159
          4       0.97      0.97      0.97       581
          5       0.67      0.94      0.78        66

avg / total       0.94      0.93      0.93      2000



In [16]:
from transformers import BertTokenizer, TFBertModel

In [17]:
import tensorflow as tf

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

In [125]:
input_ids = tf.constant(tokenizer.encode("Hell is very bad"))[None, :]  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [127]:
last_hidden_states[0].shape

TensorShape([6, 768])

In [43]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


In [19]:
def getbertEmbeddings(X):
    embedding_matrix_output = np.zeros((X.shape[0],50,768))
    for ix in range(X.shape[0]):
        sent = X['content'].iloc[ix]
        input_ids = tf.constant(tokenizer.encode(sent))[None, :]
        outputs = model(input_ids)
        last_hidden_states = outputs[0]
        last_hidden_states = last_hidden_states[0]
        for i in range(last_hidden_states.shape[0]):
            for j in range(last_hidden_states.shape[1]):
                embedding_matrix_output[ix][i][j]=last_hidden_states[i][j]    
    return embedding_matrix_output

In [20]:
data.shape

(20000, 2)

In [21]:
import numpy as np
ber_data=getbertEmbeddings(data)

KeyboardInterrupt: 

In [134]:
ber_data

array([[[ 0.05076   ,  0.16215718,  0.1385503 , ..., -0.7200436 ,
         -0.15435663,  0.12002476],
        [-0.16768497,  0.42778754,  0.76911563, ..., -0.81831127,
         -0.33187681,  0.28463736],
        [-0.08706695,  0.57004356,  0.96074969, ..., -0.27999017,
         -0.05557352, -0.42054367],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.22234818,  0.69320285, -0.15753099, ..., -0.59877563,
          0.64573443,  0.15278351],
        [-0.17078921,  0.59129208,  1.1326375 , ..., -0.60740608,
          0.36616954,  0.60645765],
        [ 0.01583041,  1.20149076,  0.27005798, ..., -0.23474357,
          0.45654729, -0.37982377],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [18]:
data_card={
    "type": "text classification",
    "file_format": "pickled dataframe",
    "sources":{
        "name":"{CARER}: Contextualized Affect Representations for Emotion Recognition",
        "Url":"https://github.com/dair-ai/emotion_dataset",
    },
    "citations":
    r'''@inproceedings{saravia-etal-2018-carer,
    title = {CARER}: Contextualized Affect Representations for Emotion Recognition",
    author = "Saravia, Elvis  and
      Liu, Hsien-Chi Toby  and
      Huang, Yen-Hao  and
      Wu, Junlin  and
      Chen, Yi-Shin",
    booktitle = "Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing",
    month = oct # "-" # nov,
    year = "2018",
    address = "Brussels, Belgium",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/D18-1404",
    doi = "10.18653/v1/D18-1404",
    pages = "3687--3697",
    abstract = "Emotions are expressed in nuanced ways, which varies by collective or individual experiences, knowledge, and beliefs. Therefore, to understand emotion, as conveyed through text, a robust mechanism capable of capturing and modeling different linguistic nuances and phenomena is needed. We propose a semi-supervised, graph-based algorithm to produce rich structural descriptors which serve as the building blocks for constructing contextualized affect representations from text. The pattern-based representations are further enriched with word embeddings and evaluated through several emotion recognition tasks. Our experimental results demonstrate that the proposed method outperforms state-of-the-art techniques on emotion recognition tasks.",
''',
    "file_location":"https://git.unbiased.cc/unbiased-intelligence-hub/emotion_detector_text/-/tree/master/Data",
    "use_cases": "Emotion recognition from text",
    "file_columns": ["text","emotion"],
    "file_size":"47.6 MB",
    "total_data_samples":"416809",
    "labels":['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
    "data_distribution":{
        'joy':'141067',
        'sadness':'121187',
        'anger':'57317',
        'fear':'47712',
        'love':'34554',
        'surprise':'14972',
    }
}

In [30]:
import json
with open('.json', 'w') as fp:
    json.dump(sample, fp)

In [22]:
model_card={
    "type": "text classification",
    "prediction_type":"multiclass classification",
    "dataset_used": "https://git.unbiased.cc/unbiased-intelligence-hub/emotion_detector_text/-/tree/master/Data",
    "output_labels": {'0':'anger','1':'fear','2':'joy','3':'love','4':'sadness','5':'surprise'},
    
    "model_unit": "LSTM",
    "used_pretrained_embeddings": 'true',
    'pretrained_embedding':'glove.6B.100d',
    "val_accuracy": '93.3',
    "usage": "emotion_recognition_text",
    "model_size": "2.89 MB",
    "input_data_type": "Tensor with a shape of (No_of_sentences,45,100)",
    "backend": "Tensorflow Keras",
    "trained_on": "8 GB RAM",
    "classification_report":{
        '0':{'precision':'0.97','recall':'0.89','f1_score':'0.93'},
        '1':{'precision':'0.92','recall':'0.88','f1_score':'0.90'},
        '2':{'precision':'0.91','recall':'0.99','f1_score':'0.95'},
        '3':{'precision':'0.99','recall':'0.67','f1_score':'0.80'},
        '4':{'precision':'0.97','recall':'0.97','f1_score':'0.97'},
        '5':{'precision':'0.67','recall':'0.94','f1_score':'0.78'},
        'overall':{'precision':'0.94','recall':'0.93','f1_score':'0.93'}
    }
}